In [0]:
 %sql

-- criando a schema Silver
CREATE SCHEMA IF NOT EXISTS silver;


In [0]:
%sql
-- Criando a tabela silver. Nesta etapa, eu estou selecionando apenas as colunas que vou utilizar, renomeando as colunas, mapeando as colunas quando há necessidade (exemplo mapeando as raças e faixas etárias)
CREATE OR REPLACE TABLE silver.enem2023_silver
USING DELTA AS 
SELECT
    NU_INSCRICAO,

-- Dimensão faixa etaria: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP

    CASE TP_FAIXA_ETARIA
    WHEN 1 THEN 'Menor de 17 anos'
    WHEN 2 THEN '17 anos'
    WHEN 3 THEN '18 anos'
    WHEN 4 THEN '19 anos'
    WHEN 5 THEN '20 anos'
    WHEN 6 THEN '21 anos'
    WHEN 7 THEN '22 anos'
    WHEN 8 THEN '23 anos'
    WHEN 9 THEN '24 anos'
    WHEN 10 THEN '25 anos'
    WHEN 11 THEN 'Entre 26 e 30 anos'
    WHEN 12 THEN 'Entre 31 e 35 anos'
    WHEN 13 THEN 'Entre 36 e 40 anos'
    WHEN 14 THEN 'Entre 41 e 45 anos'
    WHEN 15 THEN 'Entre 46 e 50 anos'
    WHEN 16 THEN 'Entre 51 e 55 anos'
    WHEN 17 THEN 'Entre 56 e 60 anos'
    WHEN 18 THEN 'Entre 61 e 65 anos'
    WHEN 19 THEN 'Entre 66 e 70 anos'
    WHEN 20 THEN 'Maior de 70 anos'
END AS FAIXA_ETARIA,
SG_UF_PROVA,


-- Dimensão sexo: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP
  CASE TP_SEXO
          WHEN 'M' THEN 'Masculino'
          WHEN 'F' THEN 'Feminino'
  END AS SEXO,

-- Dimensão raça/cor: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP    
    CASE TP_COR_RACA
        WHEN 0 THEN 'Não declarado'
        WHEN 1 THEN 'Branca'
        WHEN 2 THEN 'Preta'
        WHEN 3 THEN 'Parda'
        WHEN 4 THEN 'Amarela'
        WHEN 5 THEN 'Indígena'
        ELSE 'Não dispõe da informação'
    END AS COR_RACA,

-- Dimensão tipo de escola: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP
    CASE TP_ESCOLA
        WHEN 2 THEN 'Pública'
        WHEN 3 THEN 'Privada'
        ELSE 'Não Respondeu'
    END AS TIPO_ESCOLA,

SG_UF_ESC,

-- Presença nas provas: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP
  --Ciencias da Natureza
    CASE TP_PRESENCA_CN 
        WHEN 0 THEN 'Faltou'
        WHEN 1 THEN 'Presente'
        ELSE 'Eliminado'
    END AS PRESENCA_CN,

  --Matematica 
    CASE TP_PRESENCA_MT 
        WHEN 0 THEN 'Faltou'
        WHEN 1 THEN 'Presente'
        ELSE 'Eliminado'
    END AS PRESENCA_MT,

  --Ciencias Humanas
    CASE TP_PRESENCA_CH 
        WHEN 0 THEN 'Faltou'
        WHEN 1 THEN 'Presente'
        ELSE 'Eliminado'
    END AS PRESENCA_CH,

  --Linguagens 
    CASE TP_PRESENCA_LC 
        WHEN 0 THEN 'Faltou'
        WHEN 1 THEN 'Presente'
        ELSE 'Eliminado'
    END AS PRESENCA_LC,

  --Redacao
    CASE TP_STATUS_REDACAO
    WHEN 1	THEN 'Sem problemas'
    WHEN 2	THEN 'Anulada'
    WHEN 3	THEN 'Cópia Texto Motivador'
    WHEN 4	THEN 'Em Branco'
    WHEN 6	THEN 'Fuga ao tema'
    WHEN 7	THEN 'Não atendimento ao tipo textual'
    WHEN 8	THEN 'Texto insuficiente'
    WHEN 9	THEN 'Parte desconectada'
    END AS TP_STATUS_REDACAO,
    

-- Notas nas provas
    NU_NOTA_CN AS NOTA_CN,
    NU_NOTA_MT AS NOTA_MT,
    NU_NOTA_CH AS NOTA_CH,
    NU_NOTA_LC AS NOTA_LC,    
    NU_NOTA_REDACAO AS NOTA_REDACAO, 


--respostas ao questionariario socioeconomico que serão avaliadas na análise
-- Até que série seu pai, ou o homem responsável por você, estudou?: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP
    CASE Q001
        WHEN 'A' THEN 'Nunca estudou.'
        WHEN 'B' THEN 'Não completou a 4ª série/5º ano do Ensino Fundamental.'
        WHEN 'C' THEN 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.'
        WHEN 'D' THEN 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.'
        WHEN 'E' THEN 'Completou o Ensino Médio, mas não completou a Faculdade.'
        WHEN 'F' THEN 'Completou a Faculdade, mas não completou a Pós-graduação.'
        WHEN 'G' THEN 'Completou a Pós-graduação.'
        ELSE 'Não sei.'
    END AS ESCOLARIDADE_PAI,

-- Até que série sua mãe, ou a mulher responsável por você, estudou?: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP
    CASE Q002
        WHEN 'A' THEN 'Nunca estudou.'
        WHEN 'B' THEN 'Não completou a 4ª série/5º ano do Ensino Fundamental.'
        WHEN 'C' THEN 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.'
        WHEN 'D' THEN 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.'
        WHEN 'E' THEN 'Completou o Ensino Médio, mas não completou a Faculdade.'
        WHEN 'F' THEN 'Completou a Faculdade, mas não completou a Pós-graduação.'
        WHEN 'G' THEN 'Completou a Pós-graduação.'
        ELSE 'Não sei.'
    END AS ESCOLARIDADE_MAE,

      
-- Dimensão renda familiar: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP
    CASE Q006
        WHEN 'A' THEN 'Nenhuma Renda'
        WHEN 'B' THEN 'Até R$ 1.320,00'
        WHEN 'C' THEN 'De R$ 1.320,01 até R$ 1.980,00'
        WHEN 'D' THEN 'De R$ 1.980,01 até R$ 2.640,00'
        WHEN 'E' THEN 'De R$ 2.640,01 até R$ 3.300,00'
        WHEN 'F' THEN 'De R$ 3.300,01 até R$ 3.960,00'
        WHEN 'G' THEN 'De R$ 3.960,01 até R$ 5.280,00'
        WHEN 'H' THEN 'De R$ 5.280,01 até R$ 6.600,00'
        WHEN 'I' THEN 'De R$ 6.600,01 até R$ 7.920,00'
        WHEN 'J' THEN 'De R$ 7.920,01 até R$ 9.240,00'
        WHEN 'K' THEN 'De R$ 9.240,01 até R$ 10.560,00'
        WHEN 'L' THEN 'De R$ 10.560,01 até R$ 11.880,00'
        WHEN 'M' THEN 'De R$ 11.880,01 até R$ 13.200,00'
        WHEN 'N' THEN 'De R$ 13.200,01 até R$ 15.840,00'
        WHEN 'O' THEN 'De R$ 15.840,01 até R$ 19.800,00'
        WHEN 'P' THEN 'De R$ 19.800,01 até R$ 26.400,00'
        WHEN 'Q' THEN 'Acima de R$ 26.400,00'
    END AS RENDA_FAMILIAR

    FROM bronze.bronze_enem2023;


  


In [0]:
%sql
-- verificando a criação da tabela silver
SELECT * 
FROM silver.enem2023_silver

LIMIT 100

In [0]:
%sql
-- Avaliando a qualidade da coluna NU_INSCRICAO

SELECT NU_INSCRICAO, COUNT(*) AS TOTAL
FROM silver.enem2023_silver
GROUP BY  NU_INSCRICAO
HAVING COUNT(*) > 1

-- resultado: com a consulta a indicada, não foram encontrados valores duplicados na coluna NU_INSCRICAO.

In [0]:
%sql
-- Avaliando a qualidade da coluna NU_INSCRICAO parte 2

SELECT DISTINCT NU_INSCRICAO
FROM silver.enem2023_silver
WHERE NU_INSCRICAO IS NULL

-- resultado: não há valores nulos na coluna NU_INSCRICAO

In [0]:
%sql
--Avaliando a qualidade da coluna FAIXA_ETARIA
SELECT DISTINCT FAIXA_ETARIA
FROM silver.enem2023_silver
ORDER BY FAIXA_ETARIA

-- resultado: as faixas etárias estão dentro do esperado e estão distribuídas por faixas começando em "Menor de 17 anos" e terminando em "Maior 70 anos"


In [0]:
%sql
--Avaliando a qualidade da coluna SG_UF_PROVA

SELECT DISTINCT
SG_UF_PROVA

FROM bronze.bronze_enem2023
ORDER BY SG_UF_PROVA ASC
-- resultado: A consulta retornou as siglas referentes aos 27 estados brasileiros e o DF e nenhum valor "null". Os dados estão dentro do esperado.
-- A coluna SG_UF_PROVA será usada para a análise de desempenho por Estado, representa o estado onde foi realizada a prova.


In [0]:
%sql
--Avaliando a qualidade da coluna SEXO
SELECT DISTINCT SEXO
FROM silver.enem2023_silver

-- resultado: há apenas os valores esperados para a coluna SEXO, sendo eles "Masculino" e "Feminino".


In [0]:
%sql
--Avaliando a qualidade da coluna COR_RACA
SELECT DISTINCT COR_RACA
FROM silver.enem2023_silver

-- resultado: os valores estão dentro do esperado com as raças/cor descritas a seguir: "Amarela", "Branca", "Parda", "Preta", "Indígina" e "Não declarado"


In [0]:
%sql
--Avaliando a qualidade da coluna TIPO_ESCOLA

SELECT DISTINCT TIPO_ESCOLA

FROM silver.enem2023_silver

-- resultado: os valores estão dentro do esperado com os seguintes tipo de escola: "Pública", "Privada" e há ainda o valor "Não Respondeu" para os casos em que o aluno não informou o tipo de escola.
    

In [0]:
%sql
--Avaliando a qualidade da coluna SG_UF_ESC


SELECT  
SG_UF_ESC,
COUNT (*) AS TOTAL

FROM silver.enem2023_silver
GROUP BY SG_UF_ESC
ORDER BY TOTAL DESC

-- resultado: A consulta retornou 28 valores distintos, sendo eles os 27 estados brasileiros e o DF e valor "null". Isso pode ter ocorrido, pois talvez o preenchimento desse campo não era obrigatório.
-- considerando que a maioria das UF das escolas não preenchidas, elas não serão usadas para a análise de desempenho por Estado. Para essa análise específica, será usada a coluna "SG_UF_PROVA", que de acordo com o Dicionário de Dados, representa o estado onde foi realizada a prova.





In [0]:
%sql
-- Avaliando a qualidade das colunas referentes a presença nas provas
-- COLUNA PRESENCA_CH

SELECT DISTINCT  PRESENCA_CH
FROM silver.enem2023_silver;


-- resultado: as consultas retornaram valores dentro do esperado, são eles: "Presente", "Faltou" e "Eliminado".

In [0]:
%sql
--Avaliando a qualidade das colunas referentes a presença nas provas
--COLUNA PRESENCA_CN


SELECT DISTINCT PRESENCA_CN
FROM silver.enem2023_silver;
 

-- resultado: as consultas retornaram valores dentro do esperado, são eles: "Presente", "Faltou" e "Eliminado".

In [0]:
%sql
--Avaliando a qualidade das colunas referentes a presença nas provas
--COLUNA PRESENCA_LC

SELECT DISTINCT PRESENCA_LC
FROM silver.enem2023_silver;
 

-- resultado: as consultas retornaram valores dentro do esperado, são eles: "Presente", "Faltou" e "Eliminado".

In [0]:
%sql
--Avaliando a qualidade das colunas referentes a presença nas provas
--COLUNA PRESENCA_MT

SELECT DISTINCT PRESENCA_MT
FROM silver.enem2023_silver

-- resultado: as consultas retornaram valores dentro do esperado, são eles: "Presente", "Faltou" e "Eliminado".

In [0]:
%sql
--Avaliando a qualidade das colunas referentes a presença nas provas
--COLUNA TP_STATUS_REDACAO

SELECT 

COALESCE(TP_STATUS_REDACAO, 'Outro') AS TP_STATUS_REDACAO,
COUNT(*) AS TOTAL
FROM silver.enem2023_silver
GROUP BY TP_STATUS_REDACAO
ORDER BY TOTAL DESC

-- nao tem "5" no dicionario de dados para a coluna TP_STATUS_REDACAO, o que provavelmente gereou esses resultados nulos 

In [0]:
%sql
--Avaliando a qualidade das colunas referentes a presença nas provas
--COLUNA TP_STATUS_REDACAO - PARTE 2
SELECT DISTINCT NOTA_REDACAO
FROM silver.enem2023_silver
WHERE TP_STATUS_REDACAO IS NULL

In [0]:
%sql
--Avaliando a qualidade das colunas referentes a nota na prova de Ciências da Naturezas (NOTA_CN) - PARTE 1

SELECT  MIN(NOTA_CN), MAX(NOTA_CN)
FROM silver.enem2023_silver;
-- resultado: o menor valor encontrado foi 0 e o maior valor encontrado foi 868.4, que são valores esperados para a as notas do Enem




In [0]:
%sql
--Avaliando a qualidade das colunas referentes a nota na prova de Ciências da Naturezas (NOTA_CN) - PARTE 2

-- Avaliando a prensença de valores nulos
SELECT DISTINCT PRESENCA_CN
FROM silver.enem2023_silver
WHERE NOTA_CN IS NULL
-- resultado: valores nulos referentes a coluna NOTA_CN foram encontrados e são referentes aos cadidatos que faltaram ou foram eliminados. Por esse motivo, para a análise das notas serão considerados somente os candidatos que tiveram presença nas provas (nos dois dias de prova).

-- 




In [0]:
%sql
--Avaliando a qualidade das colunas referentes a nota na prova de Matemática (NOTA_MT) - PARTE 1

SELECT  MIN(NOTA_MT), MAX(NOTA_MT)
FROM silver.enem2023_silver;
-- resultado: o menor valor encontrado foi 0 e o maior valor encontrado foi 958.6, que são valores esperados para a as notas do Enem



In [0]:
%sql
--Avaliando a qualidade das colunas referentes a nota na prova de Matemática (NOTA_MT) - PARTE 2
-- Avaliando a prensença de valores nulos
SELECT DISTINCT PRESENCA_MT
FROM silver.enem2023_silver
WHERE NOTA_MT IS NULL
-- resultado: valores nulos referentes a coluna NOTA_MT foram encontrados e são referentes aos cadidatos que faltaram ou foram eliminados. Por esse motivo, para a análise das notas serão considerados somente os candidatos que tiveram presença nas provas (nos dois dias de prova).


In [0]:
%sql
--Avaliando a qualidade das colunas referentes a nota na prova de Ciências Humanas (NOTA_CH)

SELECT  MIN(NOTA_CH), MAX(NOTA_CH)
FROM silver.enem2023_silver;
-- resultado: o menor valor encontrado foi 0 e o maior valor encontrado foi 823, que são valores esperados para a as notas do Enem




In [0]:
%sql
--Avaliando a qualidade das colunas referentes a nota na prova de Ciências Humanas (NOTA_CH) - PARTE 2
-- Avaliando a prensença de valores nulos

SELECT DISTINCT PRESENCA_CH
FROM silver.enem2023_silver
WHERE NOTA_CH IS NULL

-- resultado: valores nulos referentes a coluna NOTA_CH foram encontrados e são referentes aos cadidatos que faltaram ou foram eliminados. Por esse motivo, para a análise das notas serão considerados somente os candidatos que tiveram presença nas provas (nos dois dias de prova).


In [0]:
%sql
--Avaliando a qualidade das colunas referentes a nota na prova de Linguagens e Códigos (NOTA_LC) - PARTE 1

SELECT  MIN(NOTA_LC), MAX(NOTA_LC)
FROM silver.enem2023_silver;
-- resultado: o menor valor encontrado foi 0 e o maior valor encontrado foi 820.8, que são valores esperados para a as notas do Enem



In [0]:
%sql
-- Avaliando a prensença de valores nulos - PARTE 2
SELECT DISTINCT PRESENCA_LC
FROM silver.enem2023_silver
WHERE NOTA_LC IS NULL
-- resultado: valores nulos referentes a coluna NOTA_LC foram encontrados e são referentes aos cadidatos que faltaram ou foram eliminados. Por esse motivo, para a análise das notas serão considerados somente os candidatos que tiveram presença nas provas (nos dois dias de prova).


In [0]:
%sql
--Avaliando a qualidade das colunas referentes a nota na prova de Linguagens e Códigos (NOTA_LC)

SELECT  MIN(NOTA_REDACAO), MAX(NOTA_REDACAO)
FROM silver.enem2023_silver;
-- resultado: o menor valor encontrado foi 0 e o maior valor encontrado foi 1000, que são valores esperados para a as notas do Enem



In [0]:
%sql
-- Avaliando a prensença de valores nulos
SELECT DISTINCT NOTA_REDACAO
FROM silver.enem2023_silver
WHERE TP_STATUS_REDACAO <> 'Sem problemas' OR TP_STATUS_REDACAO IS NULL
-- resultado: valores nulos referentes a coluna NOTA_REDACAO foram encontrados
-- conclusão: as notas da redação são válidas apenas quando o TP_STATUS_REDACAO = 'Sem problemas' e apenas esses valores serão usados para a análise das notas da redação.





In [0]:
%sql
-- Avaliando a qualidade dos dados da coluna ESCOLARIDADE_PAI

SELECT DISTINCT ESCOLARIDADE_PAI
FROM silver.enem2023_silver


-- resultado: as consultas retornaram valores dentro do esperado para a coluna ESCOLARIDADE_PAI, que correspondem aos graus de escolaridade.

In [0]:
%sql
-- Avaliando a qualidade dos dados da coluna ESCOLARIDADE_MAE	

SELECT DISTINCT ESCOLARIDADE_MAE
FROM silver.enem2023_silver

-- resultado: as consultas retornaram valores dentro do esperado para a coluna ESCOLARIDADE_MAE, que correspondem aos graus de escolaridade.

In [0]:
%sql
-- Avaliando a qualidade dos dados da coluna RENDA_FAMILIAR
SELECT DISTINCT RENDA_FAMILIAR
FROM silver.enem2023_silver
ORDER BY RENDA_FAMILIAR
-- resultado: as consultas retornaram valores dentro do esperado para a coluna RENDA_FAMILIAR, representados por valores alfanuméricos, conforme indicado no dicionário de dados do INEP e que variam de nenhuma renda até "Acima de R$ 26.400,00"



In [0]:
%sql
DESCRIBE EXTENDED silver.enem2023_silver;
-- os formatos das colunas estão dentro do esperado. Apenas o formato da coluna "NOTA_REDACAO" será alterada de INT para DOUBLE, conforme as demais notas


In [0]:
%sql
-- Recriando a tabela silver para padronizar o tipo da coluna NOTA_REDACAO com tipo DOUBLE 
-- Nessa nova versão a coluna SG_UF_ESC não será incluída

CREATE OR REPLACE TABLE silver.enem2023_silver
USING DELTA AS 
SELECT
NU_INSCRICAO,

-- Dimensão faixa etaria: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP

    CASE TP_FAIXA_ETARIA
    WHEN 1 THEN 'Menor de 17 anos'
    WHEN 2 THEN '17 anos'
    WHEN 3 THEN '18 anos'
    WHEN 4 THEN '19 anos'
    WHEN 5 THEN '20 anos'
    WHEN 6 THEN '21 anos'
    WHEN 7 THEN '22 anos'
    WHEN 8 THEN '23 anos'
    WHEN 9 THEN '24 anos'
    WHEN 10 THEN '25 anos'
    WHEN 11 THEN 'Entre 26 e 30 anos'
    WHEN 12 THEN 'Entre 31 e 35 anos'
    WHEN 13 THEN 'Entre 36 e 40 anos'
    WHEN 14 THEN 'Entre 41 e 45 anos'
    WHEN 15 THEN 'Entre 46 e 50 anos'
    WHEN 16 THEN 'Entre 51 e 55 anos'
    WHEN 17 THEN 'Entre 56 e 60 anos'
    WHEN 18 THEN 'Entre 61 e 65 anos'
    WHEN 19 THEN 'Entre 66 e 70 anos'
    WHEN 20 THEN 'Maior de 70 anos'
END AS FAIXA_ETARIA,
SG_UF_PROVA,



-- Dimensão sexo: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP
  CASE TP_SEXO
          WHEN 'M' THEN 'Masculino'
          WHEN 'F' THEN 'Feminino'
  END AS SEXO,

-- Dimensão raça/cor: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP    
    CASE TP_COR_RACA
        WHEN 0 THEN 'Não declarado'
        WHEN 1 THEN 'Branca'
        WHEN 2 THEN 'Preta'
        WHEN 3 THEN 'Parda'
        WHEN 4 THEN 'Amarela'
        WHEN 5 THEN 'Indígena'
        ELSE 'Não dispõe da informação'
    END AS COR_RACA,

-- Dimensão tipo de escola: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP
    CASE TP_ESCOLA
        WHEN 2 THEN 'Pública'
        WHEN 3 THEN 'Privada'
        ELSE 'Não Respondeu'
    END AS TIPO_ESCOLA,


-- Presença nas provas: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP
  --Ciencias da Natureza
    CASE TP_PRESENCA_CN 
        WHEN 0 THEN 'Faltou'
        WHEN 1 THEN 'Presente'
        ELSE 'Eliminado'
    END AS PRESENCA_CN,

  --Matematica 
    CASE TP_PRESENCA_MT 
        WHEN 0 THEN 'Faltou'
        WHEN 1 THEN 'Presente'
        ELSE 'Eliminado'
    END AS PRESENCA_MT,

  --Ciencias Humanas
    CASE TP_PRESENCA_CH 
        WHEN 0 THEN 'Faltou'
        WHEN 1 THEN 'Presente'
        ELSE 'Eliminado'
    END AS PRESENCA_CH,

  --Linguagens 
    CASE TP_PRESENCA_LC 
        WHEN 0 THEN 'Faltou'
        WHEN 1 THEN 'Presente'
        ELSE 'Eliminado'
    END AS PRESENCA_LC,
  --Redacao
    CASE TP_STATUS_REDACAO
    WHEN 1	THEN 'Sem problemas'
    WHEN 2	THEN 'Anulada'
    WHEN 3	THEN 'Cópia Texto Motivador'
    WHEN 4	THEN 'Em Branco'
    WHEN 6	THEN 'Fuga ao tema'
    WHEN 7	THEN 'Não atendimento ao tipo textual'
    WHEN 8	THEN 'Texto insuficiente'
    WHEN 9	THEN 'Parte desconectada'
    END AS TP_STATUS_REDACAO,



-- Notas nas provas
    NU_NOTA_CN AS NOTA_CN,
    NU_NOTA_MT AS NOTA_MT,
    NU_NOTA_CH AS NOTA_CH,
    NU_NOTA_LC AS NOTA_LC,    
    CAST(NU_NOTA_REDACAO AS  DOUBLE) AS NOTA_REDACAO, 


--respostas ao questionariario socioeconomico que serão avaliadas na análise
-- Até que série seu pai, ou o homem responsável por você, estudou?: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP
    CASE Q001
        WHEN 'A' THEN 'Nunca estudou.'
        WHEN 'B' THEN 'Não completou a 4ª série/5º ano do Ensino Fundamental.'
        WHEN 'C' THEN 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.'
        WHEN 'D' THEN 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.'
        WHEN 'E' THEN 'Completou o Ensino Médio, mas não completou a Faculdade.'
        WHEN 'F' THEN 'Completou a Faculdade, mas não completou a Pós-graduação.'
        WHEN 'G' THEN 'Completou a Pós-graduação.'
        ELSE 'Não sei.'
    END AS ESCOLARIDADE_PAI,

-- Até que série sua mãe, ou a mulher responsável por você, estudou?: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP
    CASE Q002
        WHEN 'A' THEN 'Nunca estudou.'
        WHEN 'B' THEN 'Não completou a 4ª série/5º ano do Ensino Fundamental.'
        WHEN 'C' THEN 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.'
        WHEN 'D' THEN 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.'
        WHEN 'E' THEN 'Completou o Ensino Médio, mas não completou a Faculdade.'
        WHEN 'F' THEN 'Completou a Faculdade, mas não completou a Pós-graduação.'
        WHEN 'G' THEN 'Completou a Pós-graduação.'
        ELSE 'Não sei.'
    END AS ESCOLARIDADE_MAE,

      
-- Dimensão renda familiar: mapeando os valores da conforme o dicionário de dados disponibilizado pelo INEP
    CASE Q006
        WHEN 'A' THEN 'Nenhuma Renda'
        WHEN 'B' THEN 'Até R$ 1.320,00'
        WHEN 'C' THEN 'De R$ 1.320,01 até R$ 1.980,00'
        WHEN 'D' THEN 'De R$ 1.980,01 até R$ 2.640,00'
        WHEN 'E' THEN 'De R$ 2.640,01 até R$ 3.300,00'
        WHEN 'F' THEN 'De R$ 3.300,01 até R$ 3.960,00'
        WHEN 'G' THEN 'De R$ 3.960,01 até R$ 5.280,00'
        WHEN 'H' THEN 'De R$ 5.280,01 até R$ 6.600,00'
        WHEN 'I' THEN 'De R$ 6.600,01 até R$ 7.920,00'
        WHEN 'J' THEN 'De R$ 7.920,01 até R$ 9.240,00'
        WHEN 'K' THEN 'De R$ 9.240,01 até R$ 10.560,00'
        WHEN 'L' THEN 'De R$ 10.560,01 até R$ 11.880,00'
        WHEN 'M' THEN 'De R$ 11.880,01 até R$ 13.200,00'
        WHEN 'N' THEN 'De R$ 13.200,01 até R$ 15.840,00'
        WHEN 'O' THEN 'De R$ 15.840,01 até R$ 19.800,00'
        WHEN 'P' THEN 'De R$ 19.800,01 até R$ 26.400,00'
        WHEN 'Q' THEN 'Acima de R$ 26.400,00'
    END AS RENDA_FAMILIAR

    FROM bronze.bronze_enem2023;


  


In [0]:
%sql
DESCRIBE EXTENDED silver.enem2023_silver;